In [ ]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import logging

logger = logging.getLogger()

USERNAME=""
PASSWORD=""


def login_user():
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """

    cl = Client()
    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info("Session is invalid, need to login via username and password")

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info("Attempting to login via username and password. username: %s" % USERNAME)
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
                cl.dump_settings("session.json")
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)
    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

    return cl

cl = login_user()
cl.account_info().model_dump()

In [ ]:
collections = cl.collections()
collections

**beware**, last_media_pk is misleading. collection_medias fetches all medias in the collection. Even the amount filter is only applied afterwards, so it won't optimize the query. last_media_pk is also a post-fetch filter, and it will cut the list the moment it encounters that pk. This means any non valid pk like 0 or "" will have no effect whatsoever

In [ ]:
c_col = cl.collection_medias(collection_pk="18021223057218323", amount=8161, last_media_pk="")

In [ ]:
import json, time, pydantic_core
from datetime import date
import instagrapi

def serialize_collection(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, date):
        serial = obj.isoformat()
        return serial

    if isinstance(obj, pydantic_core._pydantic_core.Url):
        return str(obj)

    if isinstance(obj, instagrapi.types.UserShort):
        return obj.__dict__

In [ ]:
with open('c_data.json', 'w') as f:
    json.dump([m.__dict__ for m in c_col], f, default=serialize_collection)

In [ ]:
with open('c_data.json', 'r') as f:
    c_col = json.load(f)

In [ ]:
c_names = [(c['user']['username'], c['user']['pk']) for c in c_col]

In [ ]:
from collections import Counter
c_names_counter = Counter(c_names)

In [ ]:
MIN_SAVED_MEDIAS = 1
c_names_filtered = list((k, v) for (k, v) in c_names_counter.items() if v > MIN_SAVED_MEDIAS)

In [ ]:
c_names_filtered = set([(id, name) for ((name, id), count) in c_names_filtered])

In [ ]:
cl.delay_range = [1, 3]

In [ ]:
for id_, name in all_ids:
    cl_tl.user_follow(id_)
    print(f"followed {name}")